# Системы хранения и обработки данных. Домашнее задание 1.
## Присяжнюк Артем Сергеевич

# Подготовка

In [1]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2 as pg

In [3]:
conn = pg.connect(dbname="postgres", host="localhost", user="postgres", password="password", port=5432)
cursor = conn.cursor()

# Задание 1

<img src="img/scheme.png">

# Задание 2

Получившаяся таблица приведена к 3НФ. 

1) Обсудим, почему таблица приведена к 1НФ. Просмотрев данные можно видеть, что ни в какой колонке таблицы нет перечисления равноценных сущностей, т.е. все атрибуты - простые, что и есть по определению 1НФ.

2) В каждой части таблицы столбцы, которые не являются ключами, зависят от первичного ключа, т.е. сгруппированы по категориям (transaction, product, customer, address). Стоит прояснить некоторые случаи. *standard_cost* является признаком **товара**, как его стандартная цена, в то время как *list_price* является признаком **транзакции**. Вместе с (1) это дает нам 2НФ.

3) Также каждый столбец, не являющийся ключом, зависит только от первичного ключа. Стоит прояснить некоторые случаи. Посмотрев на данные поймем, что столбцы *job_title* и *job_industry_category* между собой не зависимы (позиции не зависят от области). Вместе с (2) это дает нам 3НФ.

Table product {
  product_id integer [primary key]
  brand varchar
  product_line varchar
  product_class varchar
  product_size varchar
  standard_cost float
}

Table customer {
  customer_id integer [primary key]
  first_name varchar
  last_name varchar
  gender varchar
  DOB varchar
  job_title varchar
  job_industry_category varchar
  wealth_segment varchar
  deceased_indicator varchar
  owns_car bool
  address_id integer
  property_valuation integer
}

Table address {
  address_id integer [primary key]
  address varchar
  country varchar
  state varchar
  postcode integer
}

Table transaction {
  transaction_id integer [primary key]
  transaction_date varchar
  online_order bool
  order_status varchar
  list_price float
  product_id integer
  customer_id integer
}

Ref: transaction.customer_id > customer.customer_id
Ref: transaction.product_id > product.product_id
Ref: customer.address_id > address.address_id

# Задание 3

In [45]:
cursor.execute("""
               create table transaction (
                   transaction_id int NOT NULL PRIMARY KEY
                   ,transaction_date timestamp
                   ,online_order bool
                   ,order_status varchar
                   ,list_price float8
                   ,product_id int
                   ,customer_id int NOT NULL 
               )
               """)

In [46]:
cursor.execute("""
               create table product (
                   product_id integer NOT NULL PRIMARY KEY
                   ,brand varchar
                   ,product_line varchar
                   ,product_class varchar
                   ,product_size varchar
                   ,standard_cost decimal
               )
               """)

In [47]:
cursor.execute("""
               create table customer (
                   customer_id integer NOT NULL PRIMARY KEY
                   ,first_name varchar
                   ,last_name varchar
                   ,gender varchar
                   ,DOB timestamp
                   ,job_title varchar
                   ,job_industry_category varchar
                   ,wealth_segment varchar
                   ,deceased_indicator bool
                   ,owns_car bool
                   ,property_valuation integer
                   ,address_id integer NOT NULL                   
               )
               """)

In [4]:
cursor.execute("""
               create table address (
                   address_id integer NOT NULL PRIMARY KEY
                   ,address varchar
                   ,country varchar
                   ,state varchar
                   ,postcode integer              
               )
               """)

In [5]:
conn.commit()

# Задание 4

In [9]:
import pandas as pd

df_transaction = pd.read_excel("data/customer_and_transaction.xlsx", sheet_name=0)
df_transaction.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48


In [13]:
df_transaction['online_order'] = df_transaction['online_order'].apply(lambda x: x == 'True')

In [15]:
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    20000 non-null  int64         
 1   product_id        20000 non-null  int64         
 2   customer_id       20000 non-null  int64         
 3   transaction_date  20000 non-null  datetime64[ns]
 4   online_order      20000 non-null  bool          
 5   order_status      20000 non-null  object        
 6   brand             19803 non-null  object        
 7   product_line      19803 non-null  object        
 8   product_class     19803 non-null  object        
 9   product_size      19803 non-null  object        
 10  list_price        20000 non-null  float64       
 11  standard_cost     19803 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 1.7+ MB


In [182]:
df_customer = pd.read_excel("data/customer_and_transaction.xlsx", sheet_name=1)
df_customer.head()

/tmp/ipykernel_5106/1354216980.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_customer = pd.read_excel("data/customer_and_transaction.xlsx", sheet_name=1)


,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


In [183]:
df_customer['deceased_indicator'].unique(), df_customer['owns_car'].unique()

(array(['N', 'Y'], dtype=object), array(['Yes', 'No'], dtype=object))

In [184]:
df_customer['deceased_indicator'] = df_customer['deceased_indicator'].apply(lambda x: x == 'Y')
df_customer['owns_car'] = df_customer['owns_car'].apply(lambda x: x == 'Yes')

In [185]:
df_customer['gender'].unique()

array(['F', 'Male', 'Female', 'U', 'Femal', 'M'], dtype=object)

In [186]:
df_customer['gender'].replace({'Female': 'F', 'Femal': 'F', 'Male': 'M'}, inplace=True)

In [187]:
df_customer['country'].unique(), df_customer['state'].unique()

(array(['Australia'], dtype=object),
 array(['New South Wales', 'QLD', 'VIC', 'NSW', 'Victoria'], dtype=object))

In [188]:
df_customer['state'].replace({'New South Wales': 'NSW', 'Victoria': 'VIC'}, inplace=True)

In [189]:
df_customer.shape[0], df_customer['address'].nunique(), df_customer.groupby(['address', 'postcode', 'state', 'country']).count().shape[0]

(4000, 3996, 4000)

In [190]:
df_address = df_customer.groupby('address').agg({'country': lambda x: list(set(x)), 'state': lambda x: list(set(x)), 'postcode': lambda x: list(set(x))})
df_address[df_address['country'].apply(lambda x: len(x) > 1) | df_address['state'].apply(lambda x: len(x) > 1) | df_address['postcode'].apply(lambda x: len(x) > 1)]

,country,state,postcode
address,,,
3 Mariners Cove Terrace,[Australia],"[NSW, VIC]","[2216, 3108]"
3 Talisman Place,[Australia],[QLD],"[4017, 4811]"
613 Erie Lane,[Australia],[NSW],"[2088, 3996]"
64 Macpherson Junction,[Australia],"[QLD, NSW]","[2208, 4061]"


Значит для создания индекса стоит использовать все 4 столбца

In [191]:
address = df_customer.groupby(['address', 'postcode', 'state', 'country']).count().index
address2id = {address[i]: i for i in range(len(address))}
id2address = {v: k for k,v in address2id.items()}

In [192]:
df_customer.set_index(['address', 'postcode', 'state', 'country']).apply(lambda x: address2id[x.name], axis=1).values

array([ 292, 2442,   31, ..., 3346, 1364, 2530])

In [193]:
df_customer.set_index(['address', 'postcode', 'state', 'country']).apply(lambda x: address2id[x.name], axis=1).nunique()

4000

In [194]:

df_customer['address_id'] = df_customer.set_index(['address', 'postcode', 'state', 'country']).apply(lambda x: address2id[x.name], axis=1).values

In [195]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customer_id            4000 non-null   int64         
 1   first_name             4000 non-null   object        
 2   last_name              3875 non-null   object        
 3   gender                 4000 non-null   object        
 4   DOB                    3913 non-null   datetime64[ns]
 5   job_title              3494 non-null   object        
 6   job_industry_category  3344 non-null   object        
 7   wealth_segment         4000 non-null   object        
 8   deceased_indicator     4000 non-null   bool          
 9   owns_car               4000 non-null   bool          
 10  address                4000 non-null   object        
 11  postcode               4000 non-null   int64         
 12  state                  4000 non-null   object        
 13  cou

In [94]:
transaction_names = [
    'transaction_id', 'transaction_date',
    'online_order', 'order_status',
    'list_price', 
    'product_id', 'customer_id'
]
product_names = [
    'product_id', 'brand',
    'product_line', 'product_class',
    'product_size', 'standard_cost'
    
]

customer_names = [
    'customer_id', 'first_name', 'last_name',
    'gender', 'DOB',
    'job_title', 'job_industry_category',
    'wealth_segment', 'property_valuation', 'owns_car',
    'deceased_indicator',
    'address_id'
]
address_names = [
    'address_id', 'address', 'country',
    'state', 'postcode'
]

In [152]:
df_transaction.shape, df_transaction.transaction_id.nunique()

((20000, 12), 20000)

In [153]:
df_customer.shape, df_customer.customer_id.nunique()

((4000, 16), 4000)

In [238]:
mask = df_customer.DOB.apply(lambda x: x is pd.NaT)
df_customer.loc[mask, "DOB"] = pd.to_datetime(0) # because postgresql does not accept NaT

In [151]:
%pip install tqdm

  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [252]:
from tqdm import tqdm

for _, row_t in tqdm(df_transaction.iterrows()):
    cursor.execute(f"""
                insert into transaction ({','.join(transaction_names)})
                values (%s, %s, %s, %s, %s, %s, %s)
                on conflict do nothing
                """, row_t[transaction_names])

    cursor.execute(f"""
                insert into product ({','.join(product_names)})
                values (%s, %s, %s, %s, %s, %s)
                on conflict do nothing
                """, row_t[product_names])

20000it [00:35, 564.94it/s]


In [242]:
for _, row_c in tqdm(df_customer.iterrows()):
    cursor.execute(f"""
                insert into customer ({','.join(customer_names)})
                values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                on conflict do nothing
                """, row_c[customer_names])

    cursor.execute(f"""
                insert into address ({','.join(address_names)})
                values (%s, %s, %s, %s, %s)
                on conflict do nothing
                """, row_c[address_names])

4000it [00:08, 498.06it/s]


In [253]:
cursor.execute(f"""
               select * from transaction
               """)
print(cursor.fetchall())

[(1, datetime.datetime(2017, 2, 25, 0, 0), False, 'Approved', 71.49, 2, 2950), (2, datetime.datetime(2017, 5, 21, 0, 0), False, 'Approved', 2091.47, 3, 3120), (3, datetime.datetime(2017, 10, 16, 0, 0), False, 'Approved', 1793.43, 37, 402), (4, datetime.datetime(2017, 8, 31, 0, 0), False, 'Approved', 1198.46, 88, 3135), (5, datetime.datetime(2017, 10, 1, 0, 0), False, 'Approved', 1765.3, 78, 787), (6, datetime.datetime(2017, 3, 8, 0, 0), False, 'Approved', 1538.99, 25, 2339), (7, datetime.datetime(2017, 4, 21, 0, 0), False, 'Approved', 60.34, 22, 1542), (8, datetime.datetime(2017, 7, 15, 0, 0), False, 'Approved', 1292.84, 15, 2459), (9, datetime.datetime(2017, 8, 10, 0, 0), False, 'Approved', 1071.23, 67, 1305), (10, datetime.datetime(2017, 8, 30, 0, 0), False, 'Approved', 1231.15, 12, 3262), (11, datetime.datetime(2017, 1, 17, 0, 0), False, 'Approved', 574.64, 5, 1986), (12, datetime.datetime(2017, 1, 5, 0, 0), False, 'Approved', 71.16, 61, 2783), (13, datetime.datetime(2017, 2, 26, 0,

In [254]:
cursor.execute(f"""
               select * from product
               """)
print(cursor.fetchall())

[(2, 'Solex', 'Standard', 'medium', 'medium', Decimal('53.62')), (3, 'Trek Bicycles', 'Standard', 'medium', 'large', Decimal('388.92')), (37, 'OHM Cycles', 'Standard', 'low', 'medium', Decimal('248.82')), (88, 'Norco Bicycles', 'Standard', 'medium', 'medium', Decimal('381.1')), (78, 'Giant Bicycles', 'Standard', 'medium', 'large', Decimal('709.48')), (25, 'Giant Bicycles', 'Road', 'medium', 'medium', Decimal('829.65')), (22, 'WeareA2B', 'Standard', 'medium', 'medium', Decimal('45.26')), (15, 'WeareA2B', 'Standard', 'medium', 'medium', Decimal('13.44')), (67, 'Solex', 'Standard', 'medium', 'large', Decimal('380.74')), (12, 'WeareA2B', 'Standard', 'medium', 'medium', Decimal('161.6')), (5, 'Trek Bicycles', 'Mountain', 'low', 'medium', Decimal('459.71')), (61, 'OHM Cycles', 'Standard', 'low', 'medium', Decimal('56.93')), (35, 'Trek Bicycles', 'Standard', 'low', 'medium', Decimal('154.4')), (16, 'Norco Bicycles', 'Standard', 'high', 'small', Decimal('1479.11')), (79, 'Norco Bicycles', 'Sta

In [255]:
cursor.execute(f"""
               select * from customer
               """)
print(cursor.fetchall())

[(1, 'Laraine', 'Medendorp', 'F', datetime.datetime(1953, 10, 12, 0, 0), 'Executive Secretary', 'Health', 'Mass Customer', False, True, 10, 292), (2, 'Eli', 'Bockman', 'M', datetime.datetime(1980, 12, 16, 0, 0), 'Administrative Officer', 'Financial Services', 'Mass Customer', False, True, 10, 2442), (3, 'Arlin', 'Dearle', 'M', datetime.datetime(1954, 1, 20, 0, 0), 'Recruiting Manager', 'Property', 'Mass Customer', False, True, 9, 31), (4, 'Talbot', 'NaN', 'M', datetime.datetime(1961, 10, 3, 0, 0), 'NaN', 'IT', 'Mass Customer', False, False, 4, 715), (5, 'Sheila-kathryn', 'Calton', 'F', datetime.datetime(1977, 5, 13, 0, 0), 'Senior Editor', 'NaN', 'Affluent Customer', False, True, 9, 3671), (6, 'Curr', 'Duckhouse', 'M', datetime.datetime(1966, 9, 16, 0, 0), 'NaN', 'Retail', 'High Net Worth', False, True, 9, 1645), (7, 'Fina', 'Merali', 'F', datetime.datetime(1976, 2, 23, 0, 0), 'NaN', 'Financial Services', 'Affluent Customer', False, True, 4, 1992), (8, 'Rod', 'Inder', 'M', datetime.dat

In [256]:
cursor.execute(f"""
               select * from address
               """)
print(cursor.fetchall())

[(292, '060 Morning Avenue', 'Australia', 'NSW', 2016), (2442, '6 Meadow Vale Court', 'Australia', 'NSW', 2153), (31, '0 Holy Cross Court', 'Australia', 'QLD', 4211), (715, '17979 Del Mar Point', 'Australia', 'NSW', 2448), (3671, '9 Oakridge Court', 'Australia', 'VIC', 3216), (1645, '4 Delaware Trail', 'Australia', 'NSW', 2210), (1992, '49 Londonderry Lane', 'Australia', 'NSW', 2650), (3934, '97736 7th Trail', 'Australia', 'NSW', 2023), (3812, '93405 Ludington Park', 'Australia', 'VIC', 3044), (1854, '44339 Golden Leaf Alley', 'Australia', 'QLD', 4557), (867, '2 Sutherland Street', 'Australia', 'VIC', 3799), (3660, '9 Mcbride Trail', 'Australia', 'NSW', 2760), (3960, '9861 New Castle Avenue', 'Australia', 'NSW', 2428), (2134, '52 Moland Street', 'Australia', 'VIC', 3331), (3366, '82391 Kensington Lane', 'Australia', 'VIC', 3058), (383, '092 2nd Alley', 'Australia', 'NSW', 2135), (2378, '59 Spaight Circle', 'Australia', 'NSW', 2233), (207, '032 Bartelt Crossing', 'Australia', 'NSW', 244

In [257]:
conn.commit()

In [265]:
cursor.execute(f"""
               select t.transaction_id, c.customer_id, p.product_id, a.address_id 
               from transaction as t left join customer as c on t.customer_id = c.customer_id
               left join product as p on t.product_id = p.product_id 
               left join address as a on c.address_id = a.address_id
               """)
print(cursor.fetchall())

[(1, 2950, 2, 1251), (2, 3120, 3, 1355), (3, 402, 37, 314), (4, 3135, 88, 1025), (5, 787, 78, 3182), (6, 2339, 25, 2180), (7, 1542, 22, 815), (8, 2459, 15, 936), (9, 1305, 67, 2115), (10, 3262, 12, 1247), (11, 1986, 5, 1162), (12, 2783, 61, 1208), (13, 1243, 35, 3714), (14, 2717, 16, 427), (15, 247, 12, 228), (16, 2961, 3, 777), (17, 2426, 79, 2330), (18, 1842, 33, 3251), (19, 2268, 54, 2077), (20, 3002, 25, 3777), (21, 1582, 27, 1061), (22, 595, 37, 3490), (23, 2001, 37, 1623), (24, 515, 82, 1076), (25, 2822, 89, 3684), (26, 2596, 64, 1542), (27, 2666, 64, 2819), (28, 76, 19, 2860), (29, 3368, 72, 1797), (30, 1173, 91, 3476), (31, 2810, 88, 1910), (32, 3214, 1, 3357), (33, 1272, 25, 3013), (34, 2003, 99, 2120), (35, 2171, 0, 2694), (36, 3356, 92, 1827), (37, 1040, 14, 2035), (38, 2916, 2, 2922), (39, 427, 12, 2800), (40, 2448, 0, 1988), (41, 2570, 44, 656), (42, 3398, 79, 3005), (43, 488, 76, 533), (44, 165, 46, 3798), (45, 2787, 55, 513), (46, 3053, 66, 1706), (47, 475, 35, 1431), (4

# Завершение работы

In [266]:
cursor.close()
conn.close()